### Import Dependencies

In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist, euclidean
import os
import geojson as gj

### Calculate Geometric Median for precincts in LA

In [2]:
# https://stackoverflow.com/questions/30299267/geometric-median-of-multidimensional-points
# https://www.pnas.org/content/pnas/97/4/1423.full.pdf

def geometric_median(X, eps=1e-5):
    y = np.mean(X, 0)

    while True:
        D = cdist(X, [y])
        nonzeros = (D != 0)[:, 0]

        Dinv = 1 / D[nonzeros]
        Dinvs = np.sum(Dinv)
        W = Dinv / Dinvs
        T = np.sum(W * X[nonzeros], 0)

        num_zeros = len(X) - np.sum(nonzeros)
        if num_zeros == 0:
            y1 = T
        elif num_zeros == len(X):
            return y
        else:
            R = (T - y) * Dinvs
            r = np.linalg.norm(R)
            rinv = 0 if r == 0 else num_zeros/r
            y1 = max(0, 1-rinv)*T + min(1, rinv)*y

        if euclidean(y, y1) < eps:
            return y1

        y = y1

In [3]:
year = [2013,2014,2015,2016,2017,2018,2019]
filePath = '../static/data/'
fileName = f'crime_median'
outputFile = f"{filePath}{fileName}.geojson"

yearList=[]
nameList=[]
coordList=[]

for y in year:
    
    inputFile = f"{filePath}source/crime_{y}.json"
    crime_df = pd.read_json(inputFile).transpose()

    #group by precinct 
    gb = crime_df.groupby(['LAPD_Area','Year'])    
    split_crime_df = [gb.get_group(x) for x in gb.groups]

    for i, e in enumerate(split_crime_df):
        prec_crime_np = split_crime_df[i][['LAT','LON']].to_numpy()

        c = list(geometric_median(prec_crime_np))
        x = str(split_crime_df[i]["LAPD_Area_Name"].values[0])
#         print(f"{y} , {x}")

        yearList.append(y)
        nameList.append(x)
        coordList.append(c)
    

In [4]:
df = pd.DataFrame()
df["Year"] = yearList
df["LAPD_Area_Name"] = nameList
df["Coordinates"] = coordList
df[['Long','Lat']] = pd.DataFrame(df.Coordinates.tolist(), index= df.index)

In [5]:
# df.dtypes
df.head()

,Year,LAPD_Area_Name,Coordinates,Long,Lat
0,2013,Central,"[34.046268430317454, -118.25044004338203]",34.046268,-118.250440
1,2013,Rampart,"[34.06107176670108, -118.27332504902962]",34.061072,-118.273325
2,2013,Southwest,"[34.01838682642745, -118.31555108026427]",34.018387,-118.315551
3,2013,Hollenbeck,"[34.05261478030283, -118.20472845179633]",34.052615,-118.204728
4,2013,Harbor,"[33.764045318683266, -118.28659168098801]",33.764045,-118.286592


In [6]:
def data2geojson(df, filename):
    features = []
    insert_features = lambda X: features.append(
            gj.Feature(geometry=gj.Point((X["Lat"],
                                          X["Long"]
#                                           ,X["elev"]
                                         )),
                            properties=dict(lapd_area_name=X["LAPD_Area_Name"],
                                            year = X["Year"])))
    df.apply(insert_features, axis=1)
    with open(filename, 'w', encoding='utf8') as fp:
        gj.dump(gj.FeatureCollection(features), fp, sort_keys=True, ensure_ascii=False)

In [7]:
data2geojson(df, outputFile)